In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:55:45.756221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:55:45.863575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:55:45.863590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:55:45.885252: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:55:46.429578: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:55:52.207624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:55:52.208178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:55:52.208230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:55:52.208271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:55:52.208310: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 395ms/step - loss: 2.3174 - auc: 0.3517
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5615 - auc: 0.7582
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5640 - auc: 0.8403
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6645 - auc: 0.8517
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6622 - auc: 0.8676
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6239 - auc: 0.8729
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 1.5160 - auc: 0.8780
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3704 - auc: 0.8877
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2031 - auc: 0.9017
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1040 - auc: 0.9059
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0583 

1/1 [==============================] - 0s 31ms/step - loss: 0.2480 - auc: 0.9899
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2458 - auc: 0.9899
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2324 - auc: 0.9925
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2435 - auc: 0.9905
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2507 - auc: 0.9883
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2470 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2373 - auc: 0.9916
Epoch 95/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2427 - auc: 0.9896
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2432 - auc: 0.9905
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2308 - auc: 0.9928
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 1.6682 - auc: 0.6146
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5172 - auc: 0.8076
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5601 - auc: 0.8192
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4167 - auc: 0.8433
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3059 - auc: 0.8438
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1923 - auc: 0.8447
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1475 - auc: 0.8360
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1160 - auc: 0.8360
Epoch 9/100
1/1 [==============================] - 0s 44ms/step - loss: 1.0506 - auc: 0.8612
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9703 - auc: 0.8955
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2714 - auc: 0.9865
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2715 - auc: 0.9862
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2684 - auc: 0.9876
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2691 - auc: 0.9870
Epoch 92/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2654 - auc: 0.9874
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2774 - auc: 0.9855
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2548 - auc: 0.9902
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2717 - auc: 0.9870
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2747 - auc: 0.9857
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2649 - auc: 0.9876
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.6800 - auc: 0.7745
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.4577 - auc: 0.7587
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.3962 - auc: 0.7493
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2676 - auc: 0.8183
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1915 - auc: 0.8612
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1222 - auc: 0.8827
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0166 - auc: 0.9087
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9275 - auc: 0.9301
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8795 - auc: 0.9383
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.8426 - auc: 0.9452
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2364 - auc: 0.9900
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2297 - auc: 0.9915
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2333 - auc: 0.9908
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2381 - auc: 0.9898
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2272 - auc: 0.9909
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2266 - auc: 0.9915
Epoch 94/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2219 - auc: 0.9931
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2262 - auc: 0.9918
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2295 - auc: 0.9908
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2284 - auc: 0.9908
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [6]:
m='remove_Kappa'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.7299 - auc: 0.5667
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.4748 - auc: 0.8111
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4962 - auc: 0.8420
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4599 - auc: 0.8527
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3275 - auc: 0.8619
Epoch 6/100
1/1 [==============================] - 0s 40ms/step - loss: 1.2003 - auc: 0.8708
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0849 - auc: 0.8854
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0499 - auc: 0.8783
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0311 - auc: 0.8758
Epoch 10/100
1/1 [==============================] - 0s 38ms/step - loss: 0.9789 - auc: 0.8905
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.2297 - auc: 0.9929
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2401 - auc: 0.9898
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2347 - auc: 0.9902
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2287 - auc: 0.9917
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2240 - auc: 0.9930
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2344 - auc: 0.9908
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2326 - auc: 0.9912
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2347 - auc: 0.9911
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2391 - auc: 0.9903
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2372 - auc: 0.9902
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [7]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 2.3337 - auc: 0.3051
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4832 - auc: 0.7148
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4133 - auc: 0.8323
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5621 - auc: 0.8378
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6088 - auc: 0.8427
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5311 - auc: 0.8629
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.4569 - auc: 0.8670
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3451 - auc: 0.8696
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2018 - auc: 0.8776
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0778 - auc: 0.8879
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2975 - auc: 0.9826
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2940 - auc: 0.9831
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.3019 - auc: 0.9817
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2833 - auc: 0.9859
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2922 - auc: 0.9828
Epoch 93/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2905 - auc: 0.9834
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2977 - auc: 0.9815
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2870 - auc: 0.9842
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2857 - auc: 0.9855
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2938 - auc: 0.9827
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [8]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 2.7700 - auc: 0.2478
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6027 - auc: 0.6229
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4010 - auc: 0.8164
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4710 - auc: 0.8544
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6126 - auc: 0.8538
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 1.6359 - auc: 0.8574
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6118 - auc: 0.8635
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5173 - auc: 0.8724
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4004 - auc: 0.8818
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2840 - auc: 0.8838
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2789 - auc: 0.9871
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2830 - auc: 0.9865
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2643 - auc: 0.9897
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2615 - auc: 0.9903
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2606 - auc: 0.9907
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2663 - auc: 0.9891
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2686 - auc: 0.9887
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2577 - auc: 0.9902
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2673 - auc: 0.9885
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2599 - auc: 0.9901
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [9]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.6542 - auc: 0.6145
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4678 - auc: 0.8171
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4857 - auc: 0.8394
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4080 - auc: 0.8477
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2944 - auc: 0.8545
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1776 - auc: 0.8605
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1109 - auc: 0.8621
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1226 - auc: 0.8370
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0344 - auc: 0.8701
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 0.9731 - auc: 0.8937
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2633 - auc: 0.9886
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2626 - auc: 0.9882
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2682 - auc: 0.9874
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2615 - auc: 0.9891
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2631 - auc: 0.9884
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2648 - auc: 0.9879
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2768 - auc: 0.9851
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2668 - auc: 0.9880
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2604 - auc: 0.9883
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2602 - auc: 0.9895
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [10]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 345ms/step - loss: 1.5529 - auc: 0.7598
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4797 - auc: 0.7138
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3516 - auc: 0.8325
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2957 - auc: 0.8617
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1980 - auc: 0.8754
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 1.0742 - auc: 0.8962
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0115 - auc: 0.9034
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9456 - auc: 0.9197
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9077 - auc: 0.9281
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 0.8311 - auc: 0.9505
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2329 - auc: 0.9914
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2267 - auc: 0.9918
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2275 - auc: 0.9919
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2288 - auc: 0.9916
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2304 - auc: 0.9918
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2349 - auc: 0.9906
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2359 - auc: 0.9901
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2264 - auc: 0.9914
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2226 - auc: 0.9926
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2234 - auc: 0.9925
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [11]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('7feature_remove_one3.csv',index=False)
print(de)

               model       auc
0          all_model  0.947311
1  remove_AtomPairFP  0.925090
2      remove_Charge  0.946534
3       remove_Kappa  0.947149
4        remove_MAP4  0.945460
5         remove_MOE  0.945899
6    remove_MorganFP  0.946852
7    remove_Topology  0.945945
